In [1]:
import geopandas as gpd
import pandas as pd

## Open Counties data

In [2]:
file_path = "data/cb_2024_us_county_500k/cb_2024_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)
counties_gdf = counties_gdf.rename(columns={"GEOIDFQ": "AFFGEOID"})

## Get Tennessee 2023 Totals

In [3]:
population_2023_df = pd.read_csv(
    "data/ACSDP5Y2023.DP05_2025-06-20T013737/ACSDP5Y2023.DP05-Data.csv",
    low_memory=False,
)

In [4]:
population_2023_df.columns = population_2023_df.iloc[0]
population_2023_df = population_2023_df[1:]
population_2023_df = population_2023_df[
    ["Geography", "Geographic Area Name", "Estimate!!SEX AND AGE!!Total population"]
]

## Get Tennessee 2000 Totals

In [5]:
population_2000_df = pd.read_csv(
    "data/DECENNIALDPSF22000.DP1_2025-06-20T014335/DECENNIALDPSF22000.DP1-Data.csv",
    low_memory=False,
)

In [6]:
population_2000_df.columns = population_2000_df.iloc[0]
population_2000_df = population_2000_df[1:]
population_2000_df = population_2000_df[["Geography", "Number!!Total population"]]

## Merge Data

In [7]:
population_totals_df = population_2000_df.merge(
    population_2023_df, on="Geography", how="inner"
)
population_totals_df = population_totals_df.rename(
    columns={
        "Geography": "AFFGEOID",
        "Number!!Total population": "2000",
        "Estimate!!SEX AND AGE!!Total population": "2023",
    }
)

In [8]:
population_totals_df[["2000", "2023"]] = population_totals_df[["2000", "2023"]].astype(
    int
)
population_totals_df["TOTAL"] = (
    population_totals_df["2023"] - population_totals_df["2000"]
)
population_totals_df["PERCENT"] = (
    (population_totals_df["TOTAL"] / population_totals_df["2000"]) * 100
).round(decimals=2)
population_totals_df = population_totals_df.drop_duplicates("AFFGEOID").reset_index(
    drop=True
)

In [9]:
diff_gdf = counties_gdf.merge(population_totals_df, on="AFFGEOID", how="inner")

In [10]:
diff_gdf["county_id"] = diff_gdf["AFFGEOID"].str[9:]
diff_gdf["state_id"] = diff_gdf["AFFGEOID"].str[9:11]

In [17]:
state_gdf = diff_gdf[diff_gdf["STUSPS"] == "WV"]

In [18]:
state_gdf = state_gdf.to_crs(9311)
state_gdf.to_file("data/population_diff_wv.gpkg")

In [19]:
state_gdf["2023"].sum()

np.int64(1784462)

In [20]:
state_gdf["2000"].sum()

np.int64(1808344)

In [21]:
state_gdf.sort_values("PERCENT", ascending=False)

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry,2000,Geographic Area Name,2023,TOTAL,PERCENT,county_id,state_id
140,54,003,01550008,0500000US54003,54003,Berkeley,Berkeley County,WV,West Virginia,06,831770872,1057615,"POLYGON ((1846463.846 -378443.578, 1846519.264...",75905,"Berkeley County, West Virginia",126165,50260,66.21,54003,54
139,54,037,01550025,0500000US54037,54037,Jefferson,Jefferson County,WV,West Virginia,06,542072979,6150171,"POLYGON ((1865553.113 -386410.899, 1865685.385...",42190,"Jefferson County, West Virginia",58546,16356,38.77,54037,54
1783,54,061,01550037,0500000US54061,54061,Monongalia,Monongalia County,WV,West Virginia,06,932637133,14987287,"POLYGON ((1656934.514 -389302.323, 1658424.093...",81866,"Monongalia County, West Virginia",106520,24654,30.12,54061,54
1907,54,077,01558642,0500000US54077,54077,Preston,Preston County,WV,West Virginia,06,1680420301,6668159,"POLYGON ((1706222.245 -404440.508, 1706258.052...",29334,"Preston County, West Virginia",34204,4870,16.60,54077,54
719,54,065,01557950,0500000US54065,54065,Morgan,Morgan County,WV,West Virginia,06,593295507,1401571,"POLYGON ((1823675.76 -369666.797, 1823695.555 ...",14943,"Morgan County, West Virginia",17327,2384,15.95,54065,54
520,54,027,01718644,0500000US54027,54027,Hampshire,Hampshire County,WV,West Virginia,06,1658634382,10902108,"POLYGON ((1788704.013 -411597.744, 1788775.495...",20203,"Hampshire County, West Virginia",23340,3137,15.53,54027,54
519,54,031,01718557,0500000US54031,54031,Hardy,Hardy County,WV,West Virginia,06,1508190150,5624064,"POLYGON ((1787077.012 -460918.699, 1787512.619...",12669,"Hardy County, West Virginia",14236,1567,12.37,54031,54
2555,54,079,01550046,0500000US54079,54079,Putnam,Putnam County,WV,West Virginia,06,895299121,12230139,"POLYGON ((1545929.25 -542725.57, 1549139.775 -...",51589,"Putnam County, West Virginia",57250,5661,10.97,54079,54
751,54,017,01689476,0500000US54017,54017,Doddridge,Doddridge County,WV,West Virginia,06,828078406,1950541,"POLYGON ((1627043.302 -447999.725, 1628619.527...",7403,"Doddridge County, West Virginia",7767,364,4.92,54017,54
749,54,021,01689673,0500000US54021,54021,Gilmer,Gilmer County,WV,West Virginia,06,876717620,4082112,"POLYGON ((1626572.825 -503155.747, 1626804.38 ...",7160,"Gilmer County, West Virginia",7376,216,3.02,54021,54
